#### Applied Machine Learning - Mini Project 2 (Tasnim Ahmed, ta1743)

# Decision Trees

Given the fact both of the datasets have continuous features we will implement decision trees that have binary splits.

In [ ]:
import numpy as np 
import sklearn 
import pandas as pd

## Iris 

In [1]:
# has three classes and the task is to accurately predict one of the three subtypes of the Iris flower given 
# four different physical features. These features include the length and width of the sepals 
# and the petals. There is a total of 150 instances with each class having 50 instances.

In [ ]:
## Load the data

In [ ]:
## understand the data

In [ ]:
## create the decision tree class in python 

## Spambase